In [12]:
import tensorflow as tf
import numpy as np

def reset_graph(seed=42):
    tf.reset_default_graph()
    tf.set_random_seed(seed)
    np.random.seed(seed)

In [6]:
from IPython.display import clear_output, Image, display, HTML

def strip_consts(graph_def, max_const_size=32):
    """Strip large constant values from graph_def."""
    strip_def = tf.GraphDef()
    for n0 in graph_def.node:
        n = strip_def.node.add() 
        n.MergeFrom(n0)
        if n.op == 'Const':
            tensor = n.attr['value'].tensor
            size = len(tensor.tensor_content)
            if size > max_const_size:
                tensor.tensor_content = "<stripped %d bytes>"%size
    return strip_def

def show_graph(graph_def, max_const_size=32):
    """Visualize TensorFlow graph."""
    if hasattr(graph_def, 'as_graph_def'):
        graph_def = graph_def.as_graph_def()
    strip_def = strip_consts(graph_def, max_const_size=max_const_size)
    code = """
        <script>
          function load() {{
            document.getElementById("{id}").pbtxt = {data};
          }}
        </script>
        <link rel="import" href="https://tensorboard.appspot.com/tf-graph-basic.build.html" onload=load()>
        <div style="height:600px">
          <tf-graph-basic id="{id}"></tf-graph-basic>
        </div>
    """.format(data=repr(str(strip_def)), id='graph'+str(np.random.rand()))

    iframe = """
        <iframe seamless style="width:1200px;height:620px;border:0" srcdoc="{}"></iframe>
    """.format(code.replace('"', '&quot;'))
    display(HTML(iframe))

In [42]:
num_assets = 3
num_periods = 5

In [70]:
reset_graph()

with tf.name_scope('inputs'):
    X = tf.placeholder(tf.float32, shape=(1, None, num_assets, 3), name='X')
    y = tf.placeholder(tf.float32, shape=(None, num_assets+1), name='y')
    # w = tf.placeholder(tf.float32, shape=(1, num_assets, None, 3), name='w')

with tf.name_scope('cnn'):
    kernel_height1 = 3
    conv1 = tf.layers.conv2d(X, filters=2, kernel_size=(kernel_height1, 1), name='conv1')
    relu1 = tf.nn.relu(conv1)

    kernel_height2 = num_periods - kernel_width1 + 1
    conv2 = tf.layers.conv2d(relu1, filters=20, kernel_size=(kernel_height2, 1), name='conv2')
    relu2 = tf.nn.relu(conv2)

    conv3 = tf.layers.conv2d(relu2, filters=1, kernel_size=(1, 1), name='conv3')

with tf.name_scope('output'):
    cash_bias = tf.Variable([0.], name='cash_bias')

    squeezed = tf.squeeze(conv3)
    bias_broadcast = tf.expand_dims(tf.tile(cash_bias, tf.shape(squeezed)[:1]), 1)
    
    stacked = tf.concat([bias_broadcast, squeezed], axis=1, name='logits')
    output = tf.nn.softmax(stacked, name='portfolio_weights')
    
with tf.name_scope('reward'):
    trial_rewards = tf.log(tf.reduce_sum(tf.multiply(output, y), axis=1))
    total_reward = tf.reduce_sum(trial_rewards)
    
with tf.name_scope('train'):
    optimizer = tf.train.AdamOptimizer()
    training_op = optimizer.minimize(-total_reward)
    
with tf.name_scope('init_and_save'):
    init = tf.global_variables_initializer()
    saver = tf.train.Saver()

In [71]:
# show_graph(tf.get_default_graph().as_graph_def())

In [95]:
x_val = np.random.rand(1,num_periods+3,num_assets,3)
y_val = np.random.rand(4,num_assets+1)

with tf.Session() as sess:
    init.run()
    val = sess.run([output, trial_rewards, total_reward], feed_dict={X: x_val, y:y_val})

In [96]:
val

[array([[ 0.28270409,  0.22856653,  0.26901469,  0.21971476],
        [ 0.27958262,  0.24038042,  0.25847316,  0.22156383],
        [ 0.26706192,  0.2440877 ,  0.25408688,  0.23476355],
        [ 0.27921495,  0.22792666,  0.24089487,  0.25196344]], dtype=float32),
 array([-0.58605945, -0.7735647 , -0.53787613, -0.25743803], dtype=float32),
 -2.1549382]